In [ ]:
%pip install -q langchain==0.0.346 langchain_community openai==0.28.1 tiktoken==0.5.2 cohere==4.37.0 dotenv deeplake==3.9.27 wikipedia

ERROR: Cannot install langchain-community==0.0.10, langchain-community==0.0.11, langchain-community==0.0.12, langchain-community==0.0.13, langchain-community==0.0.14, langchain-community==0.0.15, langchain-community==0.0.16, langchain-community==0.0.17, langchain-community==0.0.18, langchain-community==0.0.19, langchain-community==0.0.2, langchain-community==0.0.20, langchain-community==0.0.21, langchain-community==0.0.22, langchain-community==0.0.23, langchain-community==0.0.24, langchain-community==0.0.25, langchain-community==0.0.26, langchain-community==0.0.27, langchain-community==0.0.28, langchain-community==0.0.29, langchain-community==0.0.3, langchain-community==0.0.30, langchain-community==0.0.31, langchain-community==0.0.32, langchain-community==0.0.33, langchain-community==0.0.34, langchain-community==0.0.35, langchain-community==0.0.36, langchain-community==0.0.37, langchain-community==0.0.38, langchain-community==0.0.4, langchain-community==0.0.5, langchain-community==0.0.

In [1]:
import os
from dotenv import load_dotenv


load_dotenv("../.env")
assert os.getenv("OPENAI_API_KEY")

# Document Loaders

In [8]:
from langchain.document_loaders import CSVLoader


# Load data from a CSV file using CSVLoader
loader = CSVLoader("./data/articles.csv")
documents = loader.load()

# Access the content and metadata of each document
for document in documents[:10]:    
	content = document.page_content    
	metadata = document.metadata
	print(metadata)

{'source': './data/articles.csv', 'row': 0}
{'source': './data/articles.csv', 'row': 1}
{'source': './data/articles.csv', 'row': 2}
{'source': './data/articles.csv', 'row': 3}
{'source': './data/articles.csv', 'row': 4}
{'source': './data/articles.csv', 'row': 5}
{'source': './data/articles.csv', 'row': 6}
{'source': './data/articles.csv', 'row': 7}
{'source': './data/articles.csv', 'row': 8}
{'source': './data/articles.csv', 'row': 9}


In [10]:
from langchain_community.document_loaders import WikipediaLoader


# Load content from Wikipedia using WikipediaLoader
loader = WikipediaLoader("Machine_learning")
document = loader.load()
print(document[0])

page_content='Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.\nStatistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning. \nFrom a theoretical viewpoint, probably approximatel

# Chat Model

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

In [2]:
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [5]:
messages = [
    SystemMessage(
        content="You are a helpful assistant."
    ),
    HumanMessage(
        content="What is the most popular generative AI technology in 2025?"
    ),
]

In [7]:
model_response = chat(messages)

In [8]:
type(model_response)

langchain_core.messages.ai.AIMessage

# Embedding Model

In [9]:
from langchain_community.embeddings import OpenAIEmbeddings

In [10]:
# Initialize the model
embeddings_model = OpenAIEmbeddings()

In [11]:
# Embed a list of texts
embeddings = embeddings_model.embed_documents(
    ["Hi there!", "Oh, hello!", "What's your name?", "My friends call me World", "Hello World!"]
)

In [12]:
print("Number of documents embedded:", len(embeddings))
print("Dimension of each embedding:", len(embeddings[0]))

Number of documents embedded: 5
Dimension of each embedding: 1536


# LLMChain

In [14]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


template = """List all the colors in a rainbow"""
prompt = PromptTemplate(
    template=template, input_variables=[], output_parser=StrOutputParser()
)

chat = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_chain = LLMChain(prompt=prompt, llm=chat)

llm_chain.predict()

'A rainbow typically displays seven colors, which are often remembered by the acronym ROYGBIV. The colors are:\n\n1. Red\n2. Orange\n3. Yellow\n4. Green\n5. Blue\n6. Indigo\n7. Violet\n\nThese colors appear in this order from the outer edge to the inner edge of the rainbow.'

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser

prompt = PromptTemplate.from_template(
    "List all the colors in a {item}."
)
runnable = prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0) | StrOutputParser()
runnable.invoke({"item": "rainbow"})

'A rainbow typically displays seven colors, which are often remembered by the acronym ROYGBIV. The colors are:\n\n1. Red\n2. Orange\n3. Yellow\n4. Green\n5. Blue\n6. Indigo\n7. Violet'

# Sequential

In [16]:
from langchain.prompts import PromptTemplate


post_prompt = PromptTemplate.from_template(
    """You are a business owner. Given the theme of a post, write a social media post to share on my socials.

Theme: {theme}
Content: This is social media post based on the theme above:"""
)

review_prompt = PromptTemplate.from_template(
    """You are an expert social media manager. Given the presented social media post, it is your job to write a review for the post.

Social Media Post:
{post}
Review from a Social Media Expert:"""
)

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser


llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

chain = (
    {"post": post_prompt | llm | StrOutputParser()}
    | review_prompt | llm | StrOutputParser()
)
chain.invoke({"theme": "Having a black friday sale with 50% off on everything."})

'**Review of Social Media Post: Black Friday Sale Alert**\n\nThis Black Friday sale post is a fantastic example of effective social media marketing. Here are several key strengths and a few areas for improvement:\n\n**Strengths:**\n\n1. **Engaging Visuals and Emojis:** The use of emojis (🎉🖤🛍️✨) adds a fun and festive touch, making the post visually appealing and likely to catch the eye of potential customers as they scroll through their feeds. The black heart emoji is particularly fitting for the Black Friday theme.\n\n2. **Clear and Compelling Offer:** The headline "50% OFF EVERYTHING" is bold and straightforward, immediately communicating the value of the sale. This clarity is essential for driving engagement and encouraging shares.\n\n3. **Sense of Urgency:** Phrases like "ONE DAY ONLY" and "Don’t miss out" create a sense of urgency that can motivate customers to act quickly. This is a crucial tactic for Black Friday promotions, where time-sensitive offers can lead to increased sale

In [18]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.chat_models import ChatOpenAI


llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

post_chain = post_prompt | llm | StrOutputParser()
review_chain = review_prompt | llm | StrOutputParser()
chain = {"post": post_chain} | RunnablePassthrough.assign(review=review_chain)
chain.invoke({"theme": "Having a black friday sale with 50% off on everything."})

{'post': "🎉🖤 BLACK FRIDAY SALE ALERT! 🖤🎉\n\nGet ready to shop 'til you drop because our biggest sale of the year is here! For ONE DAY ONLY, enjoy a whopping 50% OFF EVERYTHING in-store and online! 🛍️✨\n\nWhether you're treating yourself or getting a head start on holiday gifts, now's the perfect time to snag those items you've had your eye on! \n\n🗓️ Date: [Insert Date]\n🕘 Time: [Insert Time]\n🌐 Shop online: [Insert Website Link]\n\nDon't miss out on this incredible opportunity to save BIG! Tag your shopping buddy and spread the word! 🥳\n\n#BlackFriday #Sale #ShopSmall #Discounts #HolidayShopping #50Off #ShopNow",
 'review': '**Review of Social Media Post: Black Friday Sale Alert**\n\nThis Black Friday sale announcement is a well-crafted and engaging post that effectively captures attention and encourages action. Here are some key strengths and areas for improvement:\n\n**Strengths:**\n\n1. **Visual Appeal:** The use of emojis (🎉🖤🛍️✨) adds a festive and eye-catching element to the post

# Environment

In [12]:
%pip list

Package                 Version
----------------------- -----------
aioboto3                15.1.0
aiobotocore             2.24.0
aiofiles                24.1.0
aiohappyeyeballs        2.6.1
aiohttp                 3.12.15
aioitertools            0.12.0
aiosignal               1.4.0
annotated-types         0.7.0
anyio                   3.7.1
appnope                 0.1.4
asttokens               3.0.0
attrs                   25.3.0
backoff                 2.2.1
beautifulsoup4          4.13.5
boto3                   1.39.11
botocore                1.39.11
certifi                 2025.8.3
charset-normalizer      3.4.3
click                   8.2.1
cohere                  4.37
comm                    0.2.3
dataclasses-json        0.6.7
debugpy                 1.8.16
decorator               5.2.1
deeplake                3.9.27
dill                    0.4.0
distro                  1.9.0
dotenv                  0.9.9
executing               2.2.1
fastavro                1.12.0
frozenlist     